Referenced from [Langchain](https://python.langchain.com/docs/tutorials/rag/); this was enough of a RAG to figure out where the pain points are.

In [1]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma
%pip install --quiet --upgrade pypdf #TODO PyPDF prob doesn't have native code/is slow.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_community.document_loaders import PyPDFLoader
# TODO: There's a bunch of table reqs in this format; probably screws stuff up.
loader = PyPDFLoader("req_docs/20190029153.pdf") # GATEWAY SYSTEM REQUIREMENTS
docs = loader.load_and_split() # Split based on page breaks.

In [7]:
# See, the tables screw things up.
print(docs[4])

page_content='Revision:  Baseline  Document No:  DSG -RQMT -001 
Release Date:  June 27 , 2019  Page:  4 of 142 
Title:  Gateway System Requirements  
 
This document may only be released according to the Destination Control Statement on the front cover.  
 3.2.17  Crew Systems  ....................................................................................................................................... 64 
3.2.18  Extravehicular Activity  ......................................................................................................................... 66 
3.2.19 Extravehicular Robotics  ....................................................................................................................... 68 
3.2.20  Intravehicular Robotics  ........................................................................................................................ 71 
3.2.21  Utilization  ......................................................................................

Document too large, split it into chunks with overlap.

> We use the `RecursiveCharacterTextSplitter`, which will recursively split the document using common separators like new lines until each chunk is the appropriate size.

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

CHUNK_SIZE=1000
CHUNK_OVERLAP=200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)
len(all_splits)

482

It seems there are `splitters` oriented toward specific types of documents: there are loaders for "source code" and "academic dissertations".

Perhaps write something for Requirements Documents... ?

[MBET Embedding Leaderboard](https://huggingface.co/spaces/mteb/leaderboard)

In [4]:
# %pip install --quiet --upgrade langchain_voyageai
# %pip install --quiet --upgrade langchain_openai
# %pip install -qU langchain_ollama
%pip install -qU langchain-huggingface

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

print("Alright")

# from langchain_voyageai import VoyageAIEmbeddings
# from langchain_openai import OpenAIEmbeddings

# embeddings = VoyageAIEmbeddings(
#    voyage_api_key="paChL4fTAr1Rbb9YcJ1Rmo", model="voyage-2"
# )

# local_embeddings = OllamaEmbeddings(model="nomic-embed-text")
# print(local_embeddings)
# TODO: I've either got to pay $$$ for embeddings for a provider, or fetch a computer that can run ollama! :-)

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print(embeddings_model)

# I'm going to slice the splits to just 1 at this point for PoC.
# vectorstore = Chroma.from_documents(documents=all_splits[100], embedding=embeddings_model)

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached sentence_transformers-3.1.1-py3-none-any.whl.metadata (10 kB)
Using cached langchain_huggingface-0.1.0-py3-none-any.whl (20 kB)
Using cached sentence_transformers-3.1.1-py3-none-any.whl (245 kB)
Note: you may need to restart the kernel to use updated packages.
Alright


/home/ethan/Documents/code/icl-rag/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/ethan/Documents/code/icl-rag/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [10]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings_model)

In [12]:
question = "Active and Passive thermal systems should be designed with enough robustness and capacity to allow Gateway to fly in attitudes that are driven/support operations for extended periods."
sim = vectorstore.similarity_search(question)
print(len(sim))

4


In [15]:
print(sim[2].page_content)

pressure control functions, included reduction in in the internal atmosphere pressure, 
depressurization of vestibules to support departure of visiting vehicles, and airlock 
operations.  
3.2.13  Thermal  
L2-GW-0250  Thermal Subsystem Specification 
The Gateway shall comply with applicable requirements in DSG -SPEC -THR-015, Gateway 
Program Subsystem Specification for Thermal.  
Rationale:  Each Gateway Element/Module will have it s own independent Thermal Control 
Subsystem capable of maintaining its own equipment within thermal limits at all times for 
any flight attitude.  Modules within the Habitation Element will be capable of sharing thermal 
control resources with an adjacent module for flexibility during contingencies.  Each module 
may utilize active and/or passive thermal control methods to meet thermal requirements.  
The Gateway Program Thermal Control Subsystem Specification defines the specific


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# TODO: Integrate the prompts.. what are we supposed to ask here?
# TODO: Where's my inference?